In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df_bronze = spark.read.table("workspace.hemanth_bronze.emp")
df_silver = spark.read.table("workspace.hemanth_silver.emp")

In [0]:
df_bronze1  = df_bronze.withColumn("Valid_from",current_timestamp()).withColumn("Valid_to",lit(None).cast("timestamp")).withColumn("Valid_Falg",lit("Y"))

In [0]:
df_bronze1.display()

In [0]:
df = df_bronze1.alias("bronze").join(df_silver.alias("silver"),df_bronze1.empno == df_silver.empno,"left").orderBy("bronze.empno")
df.display()

In [0]:

query = f"""
        select * from workspace.hemanth_bronze.emp where empno not in (select empno from workspace.hemanth_silver.emp)
"""
df_addnewrow = spark.sql(query)
display(df_addnewrow)
display(df_addnewrow.count())

In [0]:
if df_addnewrow.count() >= 1:
        df = df_addnewrow.withColumn("Valid_from",current_timestamp()).withColumn("Valid_to",lit(None).cast("timestamp")).withColumn("Valid_Falg",lit("Y"))
        df.write.mode("append").saveAsTable("workspace.hemanth_silver.emp")
else:
     pass

In [0]:
%sql
select * from hemanth_silver.emp

In [0]:

df1 = df_bronze1.alias("bronze").join(
    df_silver.alias("silver"),
    col("bronze.empno") == col("silver.empno"),
    "inner"
).filter(
    ((col("bronze.Valid_Falg") == "Y") & (col("silver.Valid_Falg") != "N")) & (
        (col("bronze.mgr") != col("silver.mgr")) |
        (col("bronze.sal") != col("silver.sal")) |
        (col("bronze.comm") != col("silver.comm")) |
        (col("bronze.dept_id") != col("silver.dept_id"))
    )
)

df1.display()

In [0]:

if df1.count() >= 1:
        df_CustomerId = df1.select(col("bronze.empno").alias("empno"))
        empnos = [row.empno for row in df_CustomerId.select("empno").distinct().collect()]
        empnos_str = ",".join(str(e) for e in empnos)

        query = f"""
         UPDATE workspace.hemanth_silver.emp
         SET Valid_Falg = 'N',
         Valid_to = current_timestamp()
         WHERE empno in ({empnos_str})
        """

        spark.sql(query)
df_insert = df_bronze1.filter(df_bronze1.empno.isin(empnos))
df_insert = df_bronze1.filter(df_bronze1.empno.isin(empnos))
df_insert.write.mode("append").saveAsTable("workspace.hemanth_silver.emp")

In [0]:
query = f"""
        select empno from workspace.hemanth_silver.emp where empno not in (select empno from workspace.hemanth_bronze.emp)
"""
df = spark.sql(query)

if df.count() >= 1:

    empnos = [row.empno for row in df.select("empno").distinct().collect()]
    empnos_str = ",".join(str(e) for e in empnos)
    query1 = f"""
        update workspace.hemanth_silver.emp
        SET Valid_Falg = 'N',
        Valid_to = current_timestamp()
        where empno not in ({empnos})
"""
    spark.sql(query1)
        
        

In [0]:
# df_CustomerId = df1.select(col("bronze.empno").alias("empno"))
# empnos = [row.empno for row in df_CustomerId.select("empno").distinct().collect()]
# empnos_str = ",".join(str(e) for e in empnos)

# query = f"""
#     UPDATE workspace.hemanth_silver.emp
#     SET Valid_Falg = 'N',
#     Valid_to = current_timestamp()
#     WHERE empno in ({empnos_str})
# """

# spark.sql(query)
# df_insert = df_bronze1.filter(df_bronze1.empno.isin(empnos))
# df_insert = df_bronze1.filter(df_bronze1.empno.isin(empnos))
# df_insert.write.mode("append").saveAsTable("workspace.hemanth_silver.emp")



In [0]:
%sql
select * from hemanth_silver.emp order by empno

In [0]:

# df_insert = df_bronze1.filter(df_bronze1.empno.isin(empnos))
# df_insert.write.mode("append").saveAsTable("workspace.hemanth_silver.emp")